### Get files to download

In [16]:
import pandas as pd
import sys

# Load the metadata
path = "../chestx-ray/VinDr-PCXR"
df = pd.read_csv(path + "/image_labels_train.csv")

df.head()

,image_id,rad_ID,No finding,Bronchitis,Brocho-pneumonia,Other disease,Bronchiolitis,Situs inversus,Pneumonia,Pleuro-pneumonia,Diagphramatic hernia,Tuberculosis,Congenital emphysema,CPAM,Hyaline membrane disease,Mediastinal tumor,Lung tumor
0,6cb53aff85c71b98ad13d67a131708c6,R3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40414c05687cdb156823c156967b13f0,R3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0e4a464dfbf8abc6333c82f1b77b6455,R3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,f4d3fab0b71381e6b237dc36301e85a0,R3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,b58c9b1c89978a0b1f8533b7a2ca1088,R3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Potentially define a subset of specific pathologies to focus on
# Select only images with "No finding" (5143 of 7728 total obs)
df = df[df['No finding']==1.0]

# subset to only the first 500 entries 
df = df.head(500)


In [23]:
# Create the list of URLs
base_url = "https://physionet.org/files/vindr-pcxr/1.0.0/train/"
with open(path + '/download_list.txt', 'w') as f:
    for img_id in df['image_id']:
        f.write(f"{base_url}{img_id}.dicom\n")

print(f"Done! Created 'download_list.txt' with {len(df)} targeted URLs.")

Done! Created 'download_list.txt' with 500 targeted URLs.


In [24]:
import os
import requests
from tqdm import tqdm

# Deine Daten
USERNAME = 'kallepalle'
PASSWORD = 'Appl1edD33p' # Dein echtes Passwort nutzen
LIST_FILE = '../chestx-ray/VinDr-PCXR/download_list.txt'
OUTPUT_DIR = '../chestx-ray/VinDr-PCXR/train'

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

session = requests.Session()

# 1. Login-Seite aufrufen, um den CSRF-Token zu erhalten
login_url = "https://physionet.org/login/"
get_response = session.get(login_url)
csrftoken = session.cookies.get('csrftoken')

# 2. Login-Daten mit CSRF-Token vorbereiten
login_data = {
    'username': USERNAME,
    'password': PASSWORD,
    'csrfmiddlewaretoken': csrftoken,
    'next': '/content/vindr-pcxr/1.0.0/'
}

# 3. Einloggen (Referer-Header ist oft wichtig)
session.post(login_url, data=login_data, headers={'Referer': login_url})

# 4. Download-Liste einlesen
with open(LIST_FILE, 'r') as f:
    urls = [line.strip() for line in f if line.strip()]

print(f"Starte sicheren Download von {len(urls)} Dateien...")

for url in tqdm(urls):
    filename = os.path.join(OUTPUT_DIR, os.path.basename(url))
    
    # Prüfen, ob Datei schon existiert (spart Zeit bei Abbruch)
    if os.path.exists(filename):
        continue

    response = session.get(url, stream=True)
    
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024*1024): # 1MB Chunks
                f.write(chunk)
    elif response.status_code == 403:
        print(f"\nFehler 403 bei {url}. Login fehlgeschlagen oder DUA nicht aktiv.")
        break 
    else:
        print(f"\nFehler {response.status_code} bei {url}")

print("\nFertig!")

Starte sicheren Download von 500 Dateien...


100%|██████████| 500/500 [1:58:06<00:00, 14.17s/it]  


Fertig!
